In [5]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# import CRUD
from AnimalCRUD import AnimalShelter
import datetime

In [6]:
###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
username = "aacuser"
password = "aacpassword"
shelter = AnimalShelter(username, password)

# Converts a string with a date in the format of "2015-11-29".
#    This is the format used for the ACC database in documents with the key "date_of_birth".
#    Other formats are not supported.
def GetAge(datetime_string):
    try:
        current_time = datetime.datetime.now() # Get the current time
        date = datetime.datetime.strptime(datetime_string, "%Y-%m-%d")# Convert the supplied string into a date
        year_old = current_time.year - date.year
        month_old = (current_time.month - date.month) / 12 # The difference in months expressed in terms of a year
        day_old = (current_time.day - date.day) / 365 # The difference in days expressed in terms of a year
        return(round(year_old + month_old + day_old, 2))
    except:
        return 0

# This will update documents in the database with their current age and whether or not they are deceased.
#    The intent is for this query to be run on a daily basis. For an actual implementation, this process should be
#        run as a re-occuring script rather than featured here.
#    Calculating age infrequently, rather than individually at runtime, should slightly improve performance.
def update_database_age_and_living():
    deceased_outcomes = ["Euthanasia", "Died", "Disposal"]
    weeks_per_year = 52.1429
    for doc in shelter.read({}):# For every document
        dob = doc['date_of_birth']
        id = doc['_id']
        # Boolean of whether the "outcome type" of the document is within the above list
        is_deceased = (doc['outcome_type'] in deceased_outcomes)

        if is_deceased:
            if doc['is_deceased']:
                pass# Do nothing if the animal is already marked as deceased
            else:
                # Call GetAge() to calculate the age at time of death and create or update the "age field"
                #     and create or update the is_deceased field
                shelter.update_one({"_id" : id}, {"age" : (round(doc['age_upon_outcome_in_weeks'] / weeks_per_year, 2)), "is_deceased" : True})
        else:
            # Else the animal is alive; udate age and ensure is_deceased is false
            shelter.update_one({"_id" : id}, {"age" : GetAge(dob), "is_deceased" : False})
update_database_age_and_living()

# Load Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'  # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

df = pd.DataFrame.from_records(shelter.read({}))

In [7]:
#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
    ################ HEADER ################
    html.Div(className='row', style={'display': 'flex', "width": "100%"}, children=[
        html.Img(
            id='customer-image',
            src='data:image/png;base64,{}'.format(encoded_image.decode()),
            alt='customer image',
            style={'height': '10%', 'width': '10%'}),

        html.Div(style={'font-size': '32px'}, children=[
            html.Center(html.B(html.H1('Animal Shelter Database')))
        ])
    ]),

    ################ Div line ################
    html.Hr(),
    ################ Preset filtering ################
    html.Center(html.B(html.H2('Preset Filtering'))),

    html.Div(className='row', style={'display': 'flex', 'font-size': '18px', "justifyContent": "center"}, children=[
        # Radio items
        html.Div([
            dcc.RadioItems(['Water Rescue', 'Mountain Rescue', 'Disaster Rescue'], inline=True,
                           id='filter_preset_radio'),
        ]),
        # reset button
        html.Div(style={"margin-left": "5px"}, children=[
            html.Button('Reset', id='filter_preset_reset'),
        ]),
        dcc.Store(id='preset_selection', data=None),
    ]),
    ################ Div line ################
    html.Hr(),  # Div line
    html.Div([
        ################ Custom Filtering ################

        html.Center(html.B(html.H2('Custom Filtering'))),
        dcc.Store(id='filter', data={}),
        # Species Filter
        html.Div(className='row', style={'display': 'flex'}, children=[
            # Text
            html.Div([
                html.H2('Species:', style={'font-size': '18px'})
            ], style={"width": "8%"}),
            # Dropdown
            html.Div([
                dcc.Dropdown(
                    id="filter_species",
                    options=[{"label": species, "value": species} for species in df.animal_type.unique().tolist()],
                    placeholder="-Select a species-",
                    multi=True,
                    value=df.animal_type.unique().tolist())
            ], style={"width": "92%"}),
        ]),

        # Breed Filter
        html.Div(className='row', style={'display': 'flex'}, children=[
            # Text
            html.Div([
                html.H2('Breed:', style={'font-size': '18px'})
            ], style={"width": "8%"}),
            # Dropdown
            html.Div([
                dcc.Dropdown(
                    id="filter_breed",
                    options=[{"label": breed, "value": breed} for breed in df.breed.unique().tolist()],
                    placeholder="-Select a breed-",
                    multi=True)
            ], style={"width": "92%"}),
        ]),

        # Age Filter
        # NOTE: The user sees age in years, but calculations are performed in terms of weeks per client request
        html.Div(className='row', style={'display': 'flex'}, children=[
            # Text
            html.Div([
                html.H2('Age (years):', style={'font-size': '18px'})
            ], style={"width": "8%"}),
            # Dropdown
            html.Div([
                dcc.Checklist(
                    ['Exclude deceased', 'Use "age at outcome" instead of current age'],
                    ['Use "age at outcome" instead of current age'],
                    id="filter_age_options",
                ),
                dcc.RangeSlider(
                    id="filter_age",
                    min=0,
                    max=1043,
                    step=1,
                    marks={0: '0', 26: '6m', 52: '1', 78: '18m', 104: '2', 156: '3', 209: '4', 261: '5',
                           313: '6', 365: '7', 417: '8', 469: '9', 521: '10', 574: '11', 626: '12', 678: '13',
                           730: '14', 782: '15', 834: '16', 886: '17', 939: '18', 991: '19', 1042: '20+'
                           },
                    value=[0, 1043])
            ], style={"width": "92%"}),
        ]),

        # Sex Filter
        html.Div(className='row', style={'display': 'flex'}, children=[
            # Text
            html.Div([
                html.H2('Sex:', style={'font-size': '18px'})
            ], style={"width": "8%"}),
            # Dropdown
            html.Div([
                dcc.Dropdown(
                    id="filter_sex",
                    options=[{"label": sex, "value": sex} for sex in df.sex_upon_outcome.unique().tolist()],
                    placeholder="-Select a sex-",
                    multi=True,
                    value=df.sex_upon_outcome.unique().tolist())
            ], style={"width": "92%"}),
        ]),

        # Outcome Filter
        html.Div(className='row', style={'display': 'flex'}, children=[
            # Text
            html.Div([
                html.H2('Outcome:', style={'font-size': '18px'})
            ], style={"width": "8%"}),
            # Dropdown
            html.Div([
                dcc.Dropdown(
                    id="filter_outcome",
                    options=[{"label": outcome, "value": outcome} for outcome in df.outcome_type.unique().tolist()],
                    placeholder="-Select an outcome-",
                    multi=True,
                    value=df.outcome_type.unique().tolist())
            ], style={"width": "92%"}),
        ]),
    ]),

    ################ Div line ################
    html.Hr(),  # Div line
    ################ Table ################
    html.Center(html.B(html.H2("Matching Animals"))),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {
                "name": "ID",
                "id": "animal_id",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Species",
                "id": "animal_type",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Breed",
                "id": "breed",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Age",
                "id": "age",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Name",
                "id": "name",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Outcome",
                "id": "outcome_type",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Outcome Sub",
                "id": "outcome_subtype",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Age (at outcome)",
                "id": "age_upon_outcome",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Date of Birth",
                "id": "date_of_birth",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "color",
                "id": "color",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Sex",
                "id": "sex_upon_outcome",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Age (weeks)",
                "id": "age_upon_outcome_in_weeks",
                "deletable": False,
                "selectable": True
            },
            {
                "name": "Rec #",
                "id": "rec_num",
                "deletable": False,
                "selectable": True
            },

        ],
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable="single",
        row_deletable=False,
        selected_rows=[0],
        page_action="native",
        page_current=0,
        page_size=10
    ),

    ################ Div line ################
    html.Hr(),  # Div line
    ################ Charts ################

    # This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
             style={'display': 'flex'},
             children=[

                 html.Div(
                     id='graph-id',
                     className='col s12 m6',
                     style={"width": "50%"}
                 ),
                 html.Div(
                     id='map-id',
                     className='col s12 m6',
                     style={"width": "50%"}
                 )
             ]),

    ################ Div line ################
    html.Hr(),  # Div line
    ################ Footer ################
    html.H1("Rachel Aldava SNHU CS-340 Client/Server Development"),
])


#############################################
# Interaction Between Components / Controller
#############################################

# This updates the table
@app.callback(
    Output("datatable-id", "data"),
    Input("filter", "data")
)
def update_dashboard(my_filter):
    records = shelter.read(my_filter) # Read from the database
    try:# Try to display the data
        df = pd.DataFrame.from_records(records)# Load the data into pandas
        df.drop(columns=['_id'], inplace=True) # Use pandas to prune mongo-specific information
        df.fillna('', inplace=True)
    except:# If there is an error, display a blank table
        df = pd.DataFrame.from_records([])

    return df.to_dict("records")


@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    try:
        # Load the data from the table
        dff = pd.DataFrame(viewData)
        breed_counts_series = dff['breed'].value_counts() # count the number of times a breed occurs
        dff = pd.DataFrame(breed_counts_series) # convert to a data frame
        dff = dff.reset_index()
        dff.columns = ['Breed', 'Total']# rename columns
        # Since the dataframe was already sorted by order of most frequent, we can leverage this by renaming everything'
        #    after a certain index such that less common breeds will be displayed as "other" rather than have
        #    hundreds of different entries on the pie chart
        if len(dff.index) > 8:
            dff.loc[8:, 'Breed'] = 'other'

        return html.Div([
            html.H2('Breed Distribution:', style={'font-size': '18px'}),
            dcc.Graph(figure=px.pie(dff, values='Total', names='Breed'))
        ])
    except:
        return None


# This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    style_data = None
    if selected_columns != None:
        style_data = [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]
    return style_data


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    try:
        dff = pd.DataFrame.from_dict(viewData)
        # Because we only allow single row selection, the list can
        # be converted to a row index here
        if index is None:
            row = 0
        else:
            row = index[0]

        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'},
                   center=[30.75, -97.48], zoom=10, children=[
                    dl.TileLayer(id="base-layer-id"),
                    # Marker with tool tip and popup
                    # Column 13 and 14 define the grid-coordinates for
                    # the map
                    # Column 4 defines the breed for the animal
                    # Column 9 defines the name of the animal
                    dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]],
                              children=[
                                  dl.Tooltip(dff.iloc[row, 4]),
                                  dl.Popup([
                                      html.H1("Animal Name"),
                                      html.P(dff.iloc[row, 9])
                                  ])
                              ])
                ])
        ]
    except:
        return None

@app.callback(
    Output("filter_preset_radio", "value"),
    Input('filter_preset_reset', "n_clicks"))
def preset_reset(n_clicks):
    return None

# This sets the custom filter components to preset values based on the value of the filter preset radio
@app.callback(
    Output("filter_species", "value"),
    Output("filter_age", "value"),
    Output("filter_outcome", "value"),
    Output("filter_age_options", "value"),
    Output("filter_sex", "value"),
    Output("filter_breed", "value"),
    Input('filter_preset_radio', "value"))
def preset_apply(preset_value):
    # If there is no value, then display every animal
    if preset_value == None:
        filter_species = []
        filter_breed = []
        filter_outcome = []
        filter_age_options = []
        filter_sex = []
        filter_age = [0, 1043]

    # If water rescue is selected, output the relevant values to the custom filter components.
    elif "Water Rescue" in preset_value:
        filter_species = ["Dog"]
        filter_breed = ["Labrador Retriever",
                        "Labrador Retriever Mix",
                        "Chesa Bay Retr Mix",
                        "Labrador Retriever/Chesa Bay Retr",
                        "Newfoundland Mix",
                        "Newfoundland/Labrador Retriever",
                        "Labrador Retriever/Newfoundland"]
        filter_outcome = []
        filter_age_options = ['Exclude deceased', 'Use "age at outcome" instead of current age']
        filter_sex = ["Intact Female"]
        filter_age = [26, 156]

    # If mountain rescue is selected, output the relevant values to the custom filter components.
    elif "Mountain Rescue" in preset_value:
        filter_species = ["Dog"]
        filter_breed = ["German Shepherd",
                        "German Shepherd Mix",
                        "Alaskan Malamute",
                        "Alaskan Malamute Mix",
                        "German Shepherd/Alaskan Malamute",
                        "Old English Sheepdog",
                        "Siberian Husky",
                        "Siberian Husky Mix",
                        "German Shepherd/Siberian Husky",
                        "Siberian Husky/German Shepherd",
                        "Rottweiler",
                        "Rottweiler/Siberian Husky",
                        "Rottweiler Mix",
                        "Rottweiler/German Shepherd",
                        "German Shepherd/Rottweiler"]
        filter_outcome = []
        filter_age_options = ['Exclude deceased', 'Use "age at outcome" instead of current age']
        filter_sex = ["Intact Male"]
        filter_age = [26, 156]

    # If mountain rescue is selected, output the relevant values to the custom filter components.
    elif "Disaster Rescue" in preset_value:
        filter_species = ["Dog"]
        filter_breed = ["German Shepherd",
                        "German Shepherd Mix",
                        "Rottweiler",
                        "Rottweiler Mix",
                        "Rottweiler/German Shepherd",
                        "German Shepherd/Rottweiler",
                        "Doberman Pinsch",
                        "Doberman Pinsch Mix",
                        "German Shepherd/Doberman Pinsch",
                        "Bloodhound",
                        "Bloodhound Mix",
                        "Golden Retriever",
                        "Golden Retriever Mix"]
        filter_outcome = []
        filter_age_options = ['Exclude deceased', 'Use "age at outcome" instead of current age']
        filter_sex = ["Intact Male"]
        filter_age = [20, 300]

    return filter_species, filter_age, filter_outcome, filter_age_options, filter_sex, filter_breed

# This callback processes the filter components input and converts it into a valid query for PyMongo
@app.callback(
    Output("filter", "data"),
    Input("filter_species", "value"),
    Input("filter_breed", "value"),
    Input("filter_outcome", "value"),
    Input("filter_age", "value"),
    Input("filter_age_options", "value"),
    Input("filter_sex", "value"),
)
def update_filter(filter_species, filter_breed, filter_outcome, filter_age, filter_age_options, filter_sex):
    # Example query strategy:
    #
    # species_other = {"animal_type": "Other"}
    # species_bird = {"animal_type": "Bird"}
    # outcome_euthanasia = {"outcome_type" : "Euthanasia"}
    # outcome_adoption = {"outcome_type": "Adoption"}
    # query_species = {"$or": [species_other, species_bird]}
    # query_outcome = {"$or": [outcome_euthanasia, outcome_adoption]}
    # query_full = {"$and": [query_species, query_outcome]}
    # for doc in shelter.read(query_full):
    #         print(f"\n{doc}\n")
    #
    # In the above example, the query was pieced together by stacking dictionaries inside of "$or" keyed dictionaries,
    #     then keying each of those or dictionaries inside an "$and" dictionary.
    # PyMongo will read this as "any species of (other or bird) that has an outcome of (euthanasia or adoption)
    #
    #
    # We will now begin:
    filter_and = []# An empty "and" dictionary which will later add "or" dictionaries to

    # Apply species filter "or" dictionary
    if filter_species is not None and len(filter_species) > 0:
        filter_and.append({"$or": [{"animal_type": species} for species in filter_species]})

    # Apply breed filter "or" dictionary
    if filter_breed is not None and len(filter_breed) > 0:
        filter_and.append({"$or": [{"breed": breed} for breed in filter_breed]})

    # Apply outcome filter "or" dictionary
    if filter_outcome is not None and len(filter_outcome) > 0:
        filter_and.append({"$or": [{"outcome_type": outcome} for outcome in filter_outcome]})

    # Apply sex filter "or" dictionary
    if filter_sex is not None and len(filter_sex) > 0:
        filter_and.append({"$or": [{"sex_upon_outcome": sex} for sex in filter_sex]})

    # Apply deceased filter (not an or filter; there are only two possible states):
    if 'Exclude deceased' in filter_age_options:
        filter_and.append({"is_deceased": False})

    # Apply age filters
    if filter_age is not None and len(filter_age) > 0:
        # Default: we use age upon outcome, in units of weeks
        age_column = 'age_upon_outcome_in_weeks'
        age_min = filter_age[0]
        age_max = filter_age[1]

        # If option is unchecked, we use age based on date-of-birth, which is measured in years
        if 'Use "age at outcome" instead of current age' not in filter_age_options:
            age_column = 'age'
            weeks_per_year = 52.1429
            age_min /= weeks_per_year
            age_max /= weeks_per_year

        filter_and.append({age_column: {"$gt": age_min}})# for PyMongo, "$gt" means greater than
        # If there is a maximum age
        if filter_age[1] < 1040:
            filter_and.append({age_column: {"$lt": age_max}})# for PyMongo, "$lt" means lesser than

    # By default, our query will be blank
    query = {}
    # If the "and" list has any elements, we can safely add an "and" component to the search query
    if len(filter_and) != 0:
        query = {"$and": filter_and}

    return query


app.run_server(debug=True)

Dash app running on http://127.0.0.1:25662/
